# Computer Vision Quick and Dirty - CSS Summer School :)
## This notebook contains a set of quick scripts to extract interpretable image features. 
#### Features include color distribution, sharpness, faces, and objects.

It is *almost* plug-and-play, you will just have to create a couple of API keys, install a few libraries, and you are done. Just take this code and put it in a loop over all your images to extract features for your whole dataset.


##### Requirements
* OPENCV: pip install opencv-python
* Tensorflow: pip install tensorflow==1.14.0
* Google Vision API: pip install google-cloud-vision
* Numpy/Math/Matplotlib (probably you have them already)
* All files/scripts in this folder
##### API keys
* Face++: https://console.faceplusplus.com It's free ;D
* Google Vision API: create credentials from https://console.cloud.google.com/apis/credentials->create credentials-->service account keys, then save the resulting file into credentials/vision_api.json



In [ ]:

'''
importing the libraries
'''
#general
import os
import math
import json

#for matrix computations and plotting
import cv2
import numpy as np
from matplotlib import pyplot as plt

#import libraries for google Vision API
from google.cloud import vision

#importing scripts for Face++
from facepp import API
from facepp import File

#you will have to have tensorflow installed to run the code in the file below. 
import classify_image_tensorflow as tfclassify

'''
initializing context
'''
#replace this with your API keys for Face++
API_KEY = ''
API_SECRET = ''
api = API(API_KEY, API_SECRET)

#this is the environment variable with the credentials for Google Vision API
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="credentials/vision_api.json"


In [ ]:
'''
A couple of useful plotting functions - let's skip the details, this is just for us to visulize the images we are dealing with
'''

def plot_image(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #transforming to rgb for visualization
    fig = plt.gcf()
    fig.set_size_inches(10.5, 10.5)
    plt.imshow(img_rgb)
    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()

def plot_hsv(h,s,v):
    fig, (ax1, ax2,ax3) = plt.subplots(1, 3)
    fig.set_size_inches(18.5, 4)

    fig.suptitle('RGB IMAGE')
    ax1.imshow(h,cmap="hsv")
    ax1.set_title('Hue')
    ax2.imshow(s,cmap="gray")
    ax2.set_title('Saturation')
    ax3.imshow(v,cmap="gray")
    ax3.set_title('Brightness')
    plt.show()

## Extracting Colors
###  Goal: extract a color distribution in a human-readable format
Starting point: 
OPENCV, the simplest library for image analysis. By default, OpenCV reads the image in the RGB space.
![Image](https://img1.freepng.es/20180625/gpx/kisspng-rgb-color-space-rgb-color-model-light-5b30931fa1fb14.2738796215299100476635.jpg)

In [ ]:
'''
here we read and plot an image, and try to understand what the image is made of
'''
img = cv2.imread('images/Fei-Fei.jpg')
plot_image(img)
img.shape

In [ ]:
'''
the image is 3 dimensional matrix, the first 2 dimensions are width and height, the third is the depth, or color channel. 
For color images, the depth is 3, Red, Green, and Blue.
'''
img[:4,:4,:].T

Sampling RGB histograms is not very effective if we want to get interpretable features.
__The HSV Space gives a more interpretable representation.__
<img align="left" width="300" height="300" src="https://www.mathworks.com/help/images/hsvcone.gif">
<img align="center" width="300" height="300" src="http://www.texample.net/media/tikz/examples/PNG/hsv-shading.png">
The Hue value represent the pure color tone, it's a value from 0 to 360 (180 in OpenCV) where 0 is red, 30 is orange, etc.


In [ ]:
'''
let's have a look at what the hsv space looks like
'''
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #convert irgbto hsv
h,s,v = cv2.split(hsv_img) #isolating the 3 channels
plot_hsv(h,s,v)

In [ ]:
'''
What are the dominant colors of an image? 
Just count the number of pixels in the hue space belonging to each color.
'''
def extract_color_distribution(h):
    #creating a dictionary of 12 colors sampled from the hue space
    color_names=['Red','Orange','Yellow','Yellow-Green','Green','Aqua',
                 'Cyan','Azure','Blue','Violet','Magenta,','Rose'] 
    #quantize each pixel from a value in the range (180) to a value between 0 and 11
    h_quant=np.floor(np.divide(h,15)) 
    #compute distribution over these 12 values
    color_values=np.histogram(h_quant,12)[0]
    color_values=color_values/float(h.shape[0]*h.shape[1])
    #assign a label to each bin of the color distribution
    color_dict={color_names[i]:color_values[i] for i in range(len(color_values))}
    return color_dict

In [ ]:
'''
Extracting color dictionary from the hue channel of the original image
'''
color_dict=extract_color_distribution(h)
color_dict

In [ ]:
'''
Just for fun - changing hue values
'''
h_new=np.add(h,40)
hsv_img_new=hsv_img.copy()
hsv_img_new[:,:,0]=h_new
img_converted= cv2.cvtColor(hsv_img_new, cv2.COLOR_HSV2BGR)
plot_image(img_converted)

In [ ]:
img_holi = cv2.imread('images/holi.jpg')
hsv_img_holi = cv2.cvtColor(img_holi, cv2.COLOR_BGR2HSV) #convert it to hsv
h_holi,s_holi,v_holi = cv2.split(hsv_img_holi)
plot_image(img_holi)

In [ ]:
#brightness is also a good feature to distinguish between different images
[np.mean(v),  np.mean(v_holi)]

In [ ]:
[np.mean(s),  np.mean(s_holi)]

##  Computing Sharpness
#### A good proxy for Image Quality
Sharpness= definition of image edges, clarity of the shapes


<img align="left" width="600" height="300" src="https://improvephotography.com/wp-content/uploads/2014/07/sharpness-eyes-example.jpg">


In [ ]:
'''
computing image sharpness as the magnitude of the gradient computed through Sobel edge detectors
'''
def compute_sharpness(img):
    img_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dx = cv2.Sobel(img_gray,cv2.CV_64F,1,0,ksize=5)
    dy = cv2.Sobel(img_gray,cv2.CV_64F,0,1,ksize=5)
    return dx,dy

def plot_sharpness(img,dx,dy):
    plt.clf
    img_rgb=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dx = cv2.Sobel(img_gray,cv2.CV_64F,1,0,ksize=31)
    dy = cv2.Sobel(img_gray,cv2.CV_64F,0,1,ksize=31)
    fig, (ax1, ax2,ax3) = plt.subplots(1,3)
    fig.set_size_inches(20, 5)
    fig.suptitle('Edge Detection with Sobel Filter')
    ax1.imshow(img_rgb)
    ax1.set_title('Original')
    ax2.imshow(dx,cmap="gray")
    ax2.set_title('Vertical edges')
    ax3.imshow(dy,cmap="gray")
    ax3.set_title("Horizontal edges")
    plt.show()

def sharpness_feature(dx,dy):
    gradient_magnitude=cv2.magnitude(dx,dy)
    return np.sum(gradient_magnitude)/float(dx.shape[0]*dx.shape[1])

In [ ]:
#computing sharpness on Fei Fei's image
dx,dy=compute_sharpness(img)
plot_sharpness(img,dx,dy)
sharpness=sharpness_feature(dx,dy)
sharpness

In [ ]:
'''
Testing that the feature works by looking at the two halves of the image
'''
left_half=img[:,:int(img.shape[1]/float(2)),:]
right_half=img[:,int(img.shape[1]/float(2)):,:]

dxl,dyl=compute_sharpness(left_half)
dxr,dyr=compute_sharpness(right_half)
plt.clf()
fig, (ax1, ax2) = plt.subplots(1,2)
fig.set_size_inches(20, 5)
fig.suptitle('Sharpness feature')
ax1.imshow(cv2.cvtColor(left_half, cv2.COLOR_BGR2RGB))
ax1.set_title('Sharpness: '+str(sharpness_feature(dxl,dyl)))
ax2.imshow(cv2.cvtColor(right_half, cv2.COLOR_BGR2RGB))
ax2.set_title('Sharpness: '+str(sharpness_feature(dxr,dyr)))

plt.show()


## Analyzing Faces
#### Face detection, gender, age, expression classification, and face landmark localization
Why do we care about faces?
![Image](images/engageus.png)
There are a number of open source solutions available, based on tensroflow. Here we look at an existing API, Face++, that has been shown to be very accurate to detect certain properties.
Due to the training data, some of the models behind these APIs are more or less accurate depending on the demographics of the faces to be characterized.
![Image](images/gendershades.png)
Tip: when you do large-scale analysis, always double check face model biases with the gendershades dataset: http://gendershades.org


In [ ]:
'''
Calling Face++ Api. All files and libraries to generate this call as in this repo. 
Just make sure you import them, and nothing to worry about. 
The face detection function also allows for ethnicity detection. Skipping here for inclusiveness purpose.
Face++ also has more functions beyond classifying faces. 
Just check the request URL (for example: https://console.faceplusplus.com/documents/10880589) 
and change it in the detect funciton of the faceapp.py file.
'''
facedata=api.detect(image_file =File('images/Fei-Fei.jpg'),return_attributes='gender,age,smiling,eyestatus');
#this function returns a dictionary. The "faces" field  is a list of X elements where X is the number of faces detected
len(facedata["faces"])

In [ ]:
#info about the first face detected
facedata["faces"][0]

In [ ]:
#info about the second face detected
facedata["faces"][1]

## Detecting Objects
#### Everything beyond faces
There are a two ways of doing this. The open source way is to use tensorflow-based models trained on Imagenet: http://www.image-net.org/. 
Imagenet is the largest image dataset available, the visual version of imagenet, it spans thousands of categories. Generally it is used by CV researchers as a benchmark for image classifiers. The most popular versions of imagenet-based classifiers are able to distinguish between 1k objects.
The complete list of objects is here: https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a
This script uses the Inception-v3 Architecture
![Image](https://4.bp.blogspot.com/-TMOLlkJBxms/Vt3HQXpE2cI/AAAAAAAAA8E/7X7XRFOY6Xo/s1600/image03.png)
Tip: this code is using an older version of the inception model, to download the latest one, change the url in the classify_image_tensorflow.py script to: http://download.tensorflow.org/models/image/imagenet/inception-latest.tgz

In [ ]:
'''Just one line of code. This returns the top-5 predictions, i.e. the 5 output neurons and their activation score,
but let's look at the objects for which the model is most confident about.'''
plot_image(img)
tfclassify.classify('images/Fei-Fei.jpg')

In [ ]:
'''Let's try with another image'''
img_serena=cv2.imread('images/serena.jpg')
plot_image(img_serena)
tfclassify.classify('images/serena.jpg')

## Detecting absolutely everyting
#### Using Google Vision API
You have a limit of 1k requests with the free account. But Google Vision API detects face characteristics, object, scenes, and web entitites.
Similar to other models, the training data plays an important role. Vision API somehow clssifies in different ways images about the same subject coming from different places in the world.
![Image](images/bias.png)
Improvements will come thanks to the recent efforts in the community including the "Inclusive Image Challenge" https://www.kaggle.com/c/inclusive-images-challenge

In [ ]:
'''
Just prepare the request for the API, using your image path. 
You can also use image urls if the images you want to classify are in the web.
Then run the annotate_image function.
'''
req = {
    "image": {"source": {'filename': 'images/serena.jpg'}}}
client = vision.ImageAnnotatorClient()

response = client.annotate_image(req)
#let's look at the coolest part: the web entities detected
response.web_detection.web_entities

In [ ]:
response.label_annotations

In [ ]:
response.face_annotations[0]

# The End!